# **Klasifikasi Jenis Sampah dengan Xception - CNN**
---

In [1]:
# File operations and data manipulation
import os, zipfile, shutil
from google.colab import drive
from shutil import copyfile

# Random number generation and numerical computations
import random
import numpy as np

# Visualization library
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Deep learning framework and its modules
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception
from keras import layers
from keras import Model

# Ignore warnings during execution
import warnings
warnings.filterwarnings("ignore")

# Print TensorFlow version
print(tf.__version__)

2.14.0


## **Dataset Collection**
---

### Def list_directories
---

In [ ]:
def list_directories(root_path):
  for rootdir, dirs, files in os.walk(root_path):
    for subdir in dirs:
      print(os.path.join(rootdir, subdir))

### Download Data
---

In [ ]:
!wget --no-check-certificate \
  https://github.com/mrgsrylm/KlasifikasiSampah/releases/download/v1.0.0/data_organic_recyclable.zip \
  -O data_organic_recyclable.zip

!unzip data_organic_recyclable.zip

### Dataset Source
---

In [ ]:
DATASOURCE = 'DATASET'

## **Data Augmentation**
---

### Datagen from ImageDataGenerator
---

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1.0 / 255.0,
                                   zoom_range = 0.2,
                                   shear_range=0.2,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   rotation_range = 40,
                                   fill_mode = 'nearest',
                                   horizontal_flip = True,
                                   vertical_flip = True,
                                   validation_split = 0.2)

valid_datagen = ImageDataGenerator(rescale = 1.0 / 255.0,
                                   validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1.0 / 255.0)

### Datasets
---
Flow training, validation and test images in batches of 64 using datagen generator

In [ ]:
train_dataset  = train_datagen.flow_from_directory(directory = os.path.join(DATASOURCE, 'TRAIN'),
                                                   target_size = (224,224),
                                                   class_mode = 'categorical',
                                                   batch_size = 64,
                                                   subset = 'training')

validation_dataset = valid_datagen.flow_from_directory(directory = os.path.join(DATASOURCE, 'TRAIN'),
                                                  target_size = (224,224),
                                                  class_mode = 'categorical',
                                                  batch_size = 64,
                                                  subset = 'validation')

test_dataset = test_datagen.flow_from_directory(directory = os.path.join(DATASOURCE, 'TEST'),
                                             target_size = (224,224),
                                             class_mode = 'categorical',
                                             batch_size = 64)

## **Model Building**
---
Create Model using Convolutional Neural Network InceptionV3 Architecture

### Create Xception CNN Architecture (with Transfer Learning
---

In [ ]:
xception_pretrained_model = Xception(input_shape=(224, 224, 3),
                                include_top=False,
                                weights='imagenet')

for layer in xception_pretrained_model.layers:
    layer.trainable = False

xception_pretrained_model.summary()

In [ ]:
last_layer = xception_pretrained_model.get_layer('add_11')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(2, activation='sigmoid')(x)

model = Model(xception_pretrained_model.input, x)

### Add Optimizer then compile model
---

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = [tf.keras.metrics.AUC(name = 'auc')])

In [ ]:
model.summary()

## **Model Training**
---

In [ ]:
history = model.fit(train_dataset,
                    validation_data=validation_dataset,
                    epochs=20,
                    verbose=1)

## **Model Evaluation**
---

### Plotting accuracy and loss Model
---

In [ ]:
acc = history.history['au<c']
val_acc = history.history['val_auc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

# Grapich
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

axes[0].plot(epochs, acc, 'r', label='Training accuracy')
axes[0].plot(epochs, val_acc, 'b', label='Validation accuracy')
axes[0].set_title('Training and Validation accuracy')
axes[0].legend()

axes[1].plot(epochs, loss, 'r', label='Training Loss')
axes[1].plot(epochs, val_loss, 'b', label='Validation Loss')
axes[1].set_title('Training and Validation loss')
axes[1].legend()

plt.tight_layout()
plt.show()

### Validation Accuracy
---

In [ ]:
loss, accuracy = model.evaluate_generator(validation_dataset, steps=10, verbose=0)

print('Validation Accuracy: %.3f%%' % (accuracy * 100))
print('Validation Loss: %.3f%%' % (loss * 100))


### Test Accuracy
---

In [ ]:
loss, accuracy = model.evaluate(test_dataset)

print('Test Accuracy: %.3f%%' % (accuracy * 100))
print('Test Loss: %.3f%%' % (loss * 100))

## **Model Testing**
---

### Metric Score
---

### Prediction
---

In [ ]:
test_x, test_y = test_dataset.__getitem__(1)

labels = test_dataset.class_indices
labels = dict((v, k) for k, v in labels.items())

y_pred = model.predict(test_x)

plt.figure(figsize=(16, 16))
n = 16
count = 0

for i in range(n):
    plt.subplot(4, 4, i+1)
    plt.title('pred:%s / truth:%s' % (labels[np.argmax(y_pred[i])], labels[np.argmax(test_y[i])]))
    plt.imshow(test_x[i])
    if np.argmax(y_pred[i]) == np.argmax(test_y[i]):
        count += 1

accuracy = count / n * 100

print('Prediction Accuracy: {}%'.format(accuracy))

## **Model Storing**
---

### Save Model (h5)
---

In [ ]:
!mkdir -p "saved_model"
!mkdir -p "tflite_model"
# model.save('saved_model/model_xception_klasifikasi_jenis_sampah.h5')
shutil.copy('model_xception_klasifikasi_sampah.hdf5', 'saved_model/model_xception_klasifikasi_jenis_sampah.h5')

### Save Model (TFLite)
---

In [ ]:
tf.saved_model.save(model, export_dir="saved_model")

converter = tf.lite.TFLiteConverter.from_saved_model("saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_inception = converter.convert()

In [ ]:
import pathlib
tflite_model_file = pathlib.Path("tflite_model/model_xception_klasifikasi_jenis_sampah.tflite")
tflite_model_file.write_bytes(tflite_model_inception)

### Save Model to Google Drive
---

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Model
source_saved_model_inception = "/content/saved_model"
destination_saved_model_inception = "/content/drive/My Drive/klasifikasi_sampah/jenis_sampah/xception/saved_model"
shutil.copytree(source_saved_model_inception, destination_saved_model_inception)

source_tflite_model_inception = "/content/tflite_model"
destination_tflite_model_inception = "/content/drive/My Drive/klasifikasi_sampah/jenis_sampah/xception/tflite_model"
shutil.copytree(source_tflite_model_inception, destination_tflite_model_inception)

## Conclusion
---
1. Training Accuracy:
2. Validation Accuracy:
3. Testing Accuracy: